In [1]:
from distributed import Client as DaskClient
import sqlalchemy as sa

from classify.features import *

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s - %(name)s.py]: %(message)s'
)

In [3]:
DaskClient()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 117.93 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35433,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 117.93 GiB
Comm: tcp://127.0.0.1:38653,Total threads: 4
Dashboard: http://127.0.0.1:44129/status,Memory: 29.48 GiB
Nanny: tcp://127.0.0.1:34897,


In [4]:
pd.set_option('max_columns', None)

In [5]:
config = ExpungeConfig.from_yaml('classify/expunge_config.yaml')

config.years_until_next_conviction_short

[INFO - classify.config.py]: Loading config from file: classify/expunge_config.yaml


numpy.timedelta64(7,'Y')

In [6]:
query = (
    sa.select(expunge)
    .where(
        sa.or_(
            expunge.c.person_id == 1021000000606,
            expunge.c.person_id == 1070000000994
        )
    )
    .order_by(
        expunge.c.person_id, 
        expunge.c.HearingDate
    )
)

In [7]:
ddf = fetch_expunge_data(
    custom_query=query
)

ddf.head()

[INFO - classify.features.py]: Reading from table: expunge_clean


,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520


In [8]:
ddf = clean_data(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520


In [9]:
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,date_if_conviction,date_if_felony_conviction,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_short,next_conviction_disqualifier_long,from_present_disqualifier_short,from_present_disqualifier_long,is_class_1_or_2,is_class_3_or_4,class1_2,class3_4
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169,Conviction,Misdemeanor,covered elsewhere,True,NaT,2009-05-26,NaT,NaT,2011-09-08,NaN,NaN,835.0,4237,False,False,True,True,False,False,False,False,False,False
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520,Dismissed,Misdemeanor,covered elsewhere,True,2009-05-26,NaT,NaT,NaT,2011-09-08,58.0,NaN,777.0,4179,True,False,True,True,False,False,False,False,False,False
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered in 19.2-392.6 - B,True,2009-07-23,2011-09-08,NaT,NaT,2014-08-28,777.0,NaN,1085.0,3402,True,False,True,True,False,True,False,False,False,False
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered elsewhere,True,2011-09-08,2014-08-28,NaT,NaT,2018-09-18,1085.0,NaN,1482.0,2317,True,False,True,True,True,True,False,False,False,False
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520,Conviction,Felony,covered elsewhere,True,2014-08-28,2018-09-18,2018-09-18,NaT,NaT,1482.0,NaN,NaN,835,False,False,False,False,True,True,False,False,False,False


In [10]:
ddf = remove_unneeded_columns(ddf)
ddf, run_id = append_run_id(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_short,next_conviction_disqualifier_long,from_present_disqualifier_short,from_present_disqualifier_long,class1_2,class3_4,run_id
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1021000000606,2009-05-26,46.2-300,Misdemeanor,2,Guilty,NA,White,Female,169,Conviction,Misdemeanor,covered elsewhere,True,NaT,NaT,2011-09-08,NaN,NaN,835.0,4237,False,False,True,True,False,False,False,False,3dd41abc-d1c2-4ae7-b4fb-08383fb31631
1021000000606,2009-07-23,18.2-57,Misdemeanor,1,Dismissed,NA,White,Female,520,Dismissed,Misdemeanor,covered elsewhere,True,2009-05-26,NaT,2011-09-08,58.0,NaN,777.0,4179,True,False,True,True,False,False,False,False,3dd41abc-d1c2-4ae7-b4fb-08383fb31631
1021000000606,2011-09-08,18.2-250.1,Misdemeanor,U,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered in 19.2-392.6 - B,True,2009-07-23,NaT,2014-08-28,777.0,NaN,1085.0,3402,True,False,True,True,False,True,False,False,3dd41abc-d1c2-4ae7-b4fb-08383fb31631
1021000000606,2014-08-28,18.2-56.1,Misdemeanor,1,Guilty,Guilty,White,Female,520,Conviction,Misdemeanor,covered elsewhere,True,2011-09-08,NaT,2018-09-18,1085.0,NaN,1482.0,2317,True,False,True,True,True,True,False,False,3dd41abc-d1c2-4ae7-b4fb-08383fb31631
1021000000606,2018-09-18,18.2-248,Felony,U,Guilty,NA,White,Female,520,Conviction,Felony,covered elsewhere,True,2014-08-28,NaT,NaT,1482.0,NaN,NaN,835,False,False,False,False,True,True,False,False,3dd41abc-d1c2-4ae7-b4fb-08383fb31631


In [11]:
file_paths = write_to_csv(ddf)

[INFO - classify.features.py]: Expungement feature data will be written to: /tmp/expunge_data
[INFO - classify.features.py]: Clearing any data from previous runs
[INFO - classify.features.py]: Command 'rm -rf /tmp/expunge_data/expunge_features-*.csv' returned with exit value: 0
[INFO - classify.features.py]: Executing Dask task graph and writing results to CSV...
[INFO - classify.features.py]: File(s) written successfully


In [12]:
file_paths[:5]

['/tmp/expunge_data/expunge_features-0.csv']

In [13]:
load_to_db(file_paths)

[INFO - classify.features.py]: Opening connection to PostGres via Psycopg
[INFO - classify.features.py]: Loading from file: /tmp/expunge_data/expunge_features-0.csv
[INFO - classify.features.py]: Load to DB complete
